In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from glob import glob
from datetime import datetime

DATA_PATH = "./data/"
TRAIN_PATH = DATA_PATH + "train.csv"
TEST_PATH = DATA_PATH + "test.csv"
WORD_EMBED_PATH = DATA_PATH + "word_embed.txt"
CHAR_EMBED_PATH = DATA_PATH + "char_embed.txt"
QUEST_PATH = DATA_PATH + "question.csv"

train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)
question_data = pd.read_csv(QUEST_PATH)
word_embedding_data = pd.read_csv(WORD_EMBED_PATH, delimiter=" ", header=None, index_col=0)
char_embedding_data = pd.read_csv(CHAR_EMBED_PATH, delimiter=" ", header=None, index_col=0)

question_data["words"] = question_data["words"].str.split(" ")
question_data["chars"] = question_data["chars"].str.split(" ")

label = train_data["label"].values

In [2]:
from keras.preprocessing.text import Tokenizer

MAX_COUNT = 10000

word_tokenizer = Tokenizer(MAX_COUNT)
word_tokenizer.fit_on_texts(question_data["words"])

word_embedding_data = np.concatenate(
    (
        np.zeros(shape=(1, word_embedding_data.shape[1]), dtype=np.float64),
        word_embedding_data.loc[list(word_tokenizer.word_index.keys())[:MAX_COUNT]].values
    ),
    axis=0
)

char_tokenizer = Tokenizer(MAX_COUNT)
char_tokenizer.fit_on_texts(question_data["chars"])

char_embedding_data = np.concatenate(
    (
        np.zeros(shape=(1, char_embedding_data.shape[1]), dtype=np.float64),
        char_embedding_data.loc[list(char_tokenizer.word_index.keys())[:MAX_COUNT]].values
    ),
    axis=0
)

word_embedding_data.shape, char_embedding_data.shape

Using TensorFlow backend.


((10001, 300), (3049, 300))

In [3]:
from keras.preprocessing.sequence import pad_sequences

SEQ_LEN = 30

def gen_word_data(data):
    seq_word1 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q1", right_on="qid")["words"])
    seq_word2 = word_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q2", right_on="qid")["words"])
    return pad_sequences(seq_word1, maxlen=SEQ_LEN, padding="pre",truncating="pre"), \
        pad_sequences(seq_word2, maxlen=SEQ_LEN, padding="pre",truncating="pre")
    
def gen_char_data(data):
    seq_char1 = char_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q1", right_on="qid")["chars"])
    seq_char2 = char_tokenizer.texts_to_sequences(data.merge(question_data, how="left", left_on="q2", right_on="qid")["chars"])
    return pad_sequences(seq_char1, maxlen=SEQ_LEN, padding="pre",truncating="pre"), \
        pad_sequences(seq_char2, maxlen=SEQ_LEN, padding="pre",truncating="pre")

word1, word2 = gen_word_data(train_data)
char1, char2 = gen_char_data(train_data)
test_word1, test_word2 = gen_word_data(test_data)
test_char1, test_char2 = gen_char_data(test_data)

word1.shape, word2.shape, test_word1.shape, test_word2.shape, char1.shape, char2.shape, test_char1.shape, test_char2.shape

((254386, 30),
 (254386, 30),
 (172956, 30),
 (172956, 30),
 (254386, 30),
 (254386, 30),
 (172956, 30),
 (172956, 30))

In [4]:
from keras.models import Model
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam, Nadam, SGD
from keras.layers import LSTM, Bidirectional, TimeDistributed, CuDNNLSTM
from keras.layers import Conv1D, GlobalMaxPool1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dropout, BatchNormalization, Dense, Flatten, Lambda, K, Activation

In [5]:
BATCH_SIZE = 1024
NUM_EPOCHES = 50
DROP_RATE = 0.3

CONV_FILTER_LAYER1 = 128
CONV_FILTER_LAYER2 = 128

DENSE_SIZE1 = 512
DENSE_SIZE2 = 256

In [6]:
def textcnn_layer(input_tensor, kernel_size):
    conv_1 = Conv1D(filters=CONV_FILTER_LAYER1, kernel_size=kernel_size, padding="same")(input_tensor)
    conv_1 = BatchNormalization()(conv_1)
    conv_1 = Activation(activation="relu")(conv_1)
    conv_2 = Conv1D(filters=CONV_FILTER_LAYER2, kernel_size=kernel_size, padding="same")(conv_1)
    conv_2 = BatchNormalization()(conv_2)
    conv_2 = Activation(activation="relu")(conv_2)
    conv_2_max = GlobalMaxPool1D()(conv_2)
    conv_2_avg = GlobalAveragePooling1D()(conv_2)
    conv_2_merge = concatenate([conv_2_max, conv_2_avg])
    return conv_2_max

In [7]:
from sklearn.model_selection import StratifiedKFold

best_results = []
last_results = []

for i, (train_index, dev_index) in enumerate(StratifiedKFold(n_splits=10, shuffle=True).split(X=char1, y=label)):
    train_x1, train_x2, train_y = word1[train_index, :], word2[train_index, :], label[train_index]  # word/char switch
    dev_x1, dev_x2, dev_y = word1[dev_index, :], word2[dev_index, :], label[dev_index]  # word/char switch
    
    input1 = Input(shape=(SEQ_LEN,), dtype="int32")
    input2 = Input(shape=(SEQ_LEN,), dtype="int32")

    embedding_layer = Embedding(
        input_dim=word_embedding_data.shape[0],  # word/char switch
        output_dim=word_embedding_data.shape[1],  # word/char switch
        weights=[word_embedding_data],  # word/char switch
        input_length=SEQ_LEN,
        trainable=False
    )
    
    vector1 = embedding_layer(input1)
    vector2 = embedding_layer(input2)
    
    conv1a, conv1b = textcnn_layer(vector1, kernel_size=1), textcnn_layer(vector2, kernel_size=1)
    conv2a, conv2b = textcnn_layer(vector1, kernel_size=2), textcnn_layer(vector2, kernel_size=2)
    conv3a, conv3b = textcnn_layer(vector1, kernel_size=3), textcnn_layer(vector2, kernel_size=3)
    conv4a, conv4b = textcnn_layer(vector1, kernel_size=4), textcnn_layer(vector2, kernel_size=4)
    conv5a, conv5b = textcnn_layer(vector1, kernel_size=5), textcnn_layer(vector2, kernel_size=5)
    conv6a, conv6b = textcnn_layer(vector1, kernel_size=6), textcnn_layer(vector2, kernel_size=6)
    
    merge_a = concatenate([conv1a, conv2a, conv3a, conv4a, conv5a, conv6a])
    merge_b = concatenate([conv1b, conv2b, conv3b, conv4b, conv5b, conv6b])
    diff = Lambda(lambda x: K.abs(x[0] - x[1]))([merge_a, merge_b])
    mult = Lambda(lambda x: x[0] * x[1])([merge_a, merge_b])
    merge = concatenate([diff, mult])
    
    x = Dropout(DROP_RATE)(merge)
    x = BatchNormalization()(x)
    x = Dense(DENSE_SIZE1, activation="relu")(x)
    x = Dropout(DROP_RATE)(x)
    x = BatchNormalization()(x)
    x = Dense(DENSE_SIZE2, activation="relu")(x)
    x = Dropout(DROP_RATE)(x)
    x = BatchNormalization()(x)
    pred = Dense(1, activation="sigmoid")(x)
    
    model = Model(inputs=[input1, input2], outputs=pred)
    model.compile(
        optimizer="nadam",
        loss="binary_crossentropy",
        metrics=["acc"]
    )
    
    early_stopping = EarlyStopping("val_loss", patience=10)
    check_point = ModelCheckpoint(
        "./log/%s.TextCNN.word.{epoch:03d}.hdf5" % (datetime.now().strftime("%Y%m%d-%H%M%S")),  # word/char switch
        monitor="val_loss",
        save_best_only=True,
    )
    
    fit_res = model.fit(
        x=[train_x1, train_x2],
        y=train_y,
        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHES,
        validation_data=([dev_x1, dev_x2], dev_y),
        shuffle=True,
        callbacks=[early_stopping, check_point]
    )
    
    pred_last = model.predict([test_word1, test_word2], batch_size=BATCH_SIZE)  # word/char switch
    last_results.append(pd.DataFrame(pred_last, columns=["y_pre"]))
    
    print("load model %s" % (glob("./log/*.hdf5")[-1].replace("\\", "/"),))
    model.load_weights(glob("./log/*.hdf5")[-1].replace("\\", "/"))
    pred_best = model.predict([test_word1, test_word2], batch_size=BATCH_SIZE)  # word/char switch
    best_results.append(pd.DataFrame(pred_best, columns=["y_pre"]))

pd.DataFrame(pd.concat(last_results, axis=1).mean(axis=1), columns=["y_pre"]).to_csv(
    "./result/%s-TextCNN_word_last.csv" % (datetime.now().strftime("%Y%m%d-%H%M%S")),  # word/char switch
    index=False
)
pd.DataFrame(pd.concat(best_results, axis=1).mean(axis=1), columns=["y_pre"]).to_csv(
    "./result/%s-TextCNN_word_best.csv" % (datetime.now().strftime("%Y%m%d-%H%M%S")),  # word/char switch
    index=False
)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Train on 228946 samples, validate on 25440 samples
Epoch 1/50
228946/228946 [==============================] - 99s 433us/step - loss: 0.4382 - acc: 0.7953 - val_loss: 0.3387 - val_acc: 0.8498
Epoch 2/50
228946/228946 [==============================] - 90s 392us/step - loss: 0.3082 - acc: 0.8647 - val_loss: 0.2845 - val_acc: 0.8755
Epoch 3/50
228946/228946 [==============================] - 93s 405us/step - loss: 0.2626 - acc: 0.8868 - val_loss: 0.2673 - val_acc: 0.8857
Epoch 4/50
228946/228946 [==============================] - 92s 402us/step - loss: 0.2304 - acc: 0.9020 - val_loss: 0.2649 - val_acc: 0.8886
Epoch 5/50
228946/228946 [==============================] - 89s 391us/step - loss: 0.2048 - acc: 0.9137 - val_loss: 0.2551 - val_acc: 0.8915
Epoch 6/50
228946/228946 [==============================] - 92s 400us/step - loss: 0.1845 - acc: 0.9231 - val_loss: 0.2518 - val_acc: 0.8948
Epoch 7/50
228946/22

KeyboardInterrupt: 